In [1]:
import re
import numpy as np
import pandas as pd
import nltk
import random
from random import randrange


In [ ]:
nltk.download('punkt')

In [ ]:
text2 = "The final product of the original study is a published manuscript in the journal Physics of Fluids. Although the manuscript is well detailed, the code and input data used to produce the computational results were not made publicly available by the authors. In that regard, we consider the study to not be reproducible. Thus, we aim to replicate the scientific findings claimed in the original study with our own research software stack and deliver reproducible results. PetIBM10 is developed in the open under the permissive (non-copyleft) 3-Clause BSD license, version-controlled with Git, and hosted on a public GitHub repository.2 Each major release of the software is archived on the data repository Zenodo. Our implementation of the three-dimensional rolling and pitching wing, which relies on PetIBM, is also open source and available on GitHub3 under the same license. The repository contains all input data and processing scripts that were used to produce the computational results reported in the next section. This allows anyone to inspect the code, to verify the steps that were taken to produce computational results, and to modify and re-use it for other applications. The repository also includes README files to guide readers that may be interested in re-running the analysis. Upon submission of the present manuscript, the application repository, as well as the data needed to reproduce the figures, have been archived on Zenodo. We leveraged our University high-performance-computing (HPC) cluster, called Pegasus, to run all simulations reported here. (We used computational nodes with Dual 20-Core 3.70GHz Intel Xeon Gold 6148 processors and NVIDIA V100 GPU devices.) To reduce the burden of building PetIBM and its applications on the cluster, we used the container technology from Docker11 and Singularity.12 Containers allow us to capture the conditions of analysis in a formatted image that can be shared with others. We have already used Docker containers in the past to create a reproducible workflow for scientific applications on the public cloud provider Microsoft Azure.13 Here, we aim to adopt a similar workflow on our local HPC cluster. Early in this replication study, we hit a snag: Docker is not available to users on Pegasus. Indeed, Docker is not available at most HPC centers for security reasons. Submitting container-based jobs with Docker implies running a Docker daemon (a background process) that requires root privileges that users do not and should not have on shared production clusters. Thus, we decided to leverage the Singularity container technology to conduct the replication study on Pegasus. Singularity is more recent than Docker, was designed from the ground up to prevent escalation of user privileges, and is compatible with Docker images. Our reproducible workflow starts with creating a Docker image that installs PetIBM and its applications, as well as all their dependencies. We then push the image to a public registry on DockerHub.4 Anyone interested in using the application code can now pull the image from the registry and spin up a Docker container to get a faithfully reproduced computational environment. Next, we use the cloud service Singularity Hub to build a"
text = "production, wake topology, and propulsive performance of a pitching and rolling wing. Although our numerical values do not fully match those from the original study Li and Dong5, we obtain the same trends and thus consider this replication attempt to be successful. A CFD solver typically outputs the solution of primary variables. For example, PetIBM outputs the pressure and velocity fields, as well as the body forces. We often use multiple post-processing scripts to generate the final data and figures reported and analyzed in the manuscript; it involves computing secondary data, such as the vorticity field, the aerodynamic power and forces. If the code is not made available, readers cannot inspect what has been done to produce these data; bugs introduced in these post-processing steps would go undetected. If no code is available, we cannot explain discrepancies observed between our replication and the original study. As Donoho and coworkers14 once said: “The only way we’d ever get to the bottom of such a discrepancy is if we both worked reproducibly and studied detailed differences between code and data.” We made our best efforts to ensure that our replication study is reproducible. Our computational application makes use of fully open-source tools, and we created a GitHub repository6 for this study. The repository contains the source code of the PetIBM application, as well as all input files of the simulations reported here, and pre- and postprocessing Python scripts. We adopted a reproducible workflow to run computational simulations; it makes use of Docker images and Singularity recipes to capture the computational environment. With Singularity, we ran container-based jobs on our universitymanaged HPC cluster. The GitHub repository also contains the job-submission scripts that were use to run the simulations on our cluster; they can be adapted to run on other platforms if readers are interested in reproducing our results. Admittedly, not everyone has access to an HPC cluster with GPU nodes and with Singularity installed. Lacking those resources, it becomes difficult to fully reproduce our workflow. However, we made the effort to deposit on Zenodo7 the primary data (directly output from our CFD solver) and post-processing scripts needed to reproduce the figures of the present manuscript. Once the Zenodo repository is downloaded, readers should be able to spin up a Docker container and run a Bash script to compute the secondary data and generate the figures, or generate different figures to explore the data in new ways. The Docker images produced and used for this study are stored on DockerHub8, under a basic free subscription. In the event Docker adopts a policy to automatically purge inactive images (those who have not been recently downloaded) from the Hub, the Dockerfiles are version-controlled on the GitHub repository and can be used to rebuild the images. We spent time engineering a transparent and reproducible workflow to produce the artifacts of this replication study. Surely, we cannot assert our steps will be fully reproducible in years from now; the software stack could very well become obsolete with new hardware generations. While the likelihood of the study being reproducible may decrease with the years, the transparency of the steps we took to generate the data shall remain constant."

In [2]:
random.seed(9)
text = ". ".join([f"{str(s)} "*randrange(1, 8) for s in np.arange(0, 45)])
print(len(nltk.word_tokenize(text)))
text

217


'0 0 0 0 . 1 1 1 1 1 . 2 2 2 . 3 3 3 . 4 4 . 5 5 . 6 6 6 6 6 6 6 . 7 7 7 7 7 7 . 8 . 9 9 9 . 10 10 10 10 10 . 11 11 11 11 . 12 12 12 12 12 . 13 . 14 14 14 . 15 15 15 15 15 . 16 16 16 16 16 . 17 17 17 17 17 17 . 18 . 19 19 19 19 19 19 . 20 20 20 20 . 21 21 . 22 22 22 22 22 22 . 23 23 23 23 . 24 24 24 24 24 24 . 25 25 25 25 . 26 26 . 27 27 . 28 28 . 29 . 30 . 31 31 . 32 32 32 32 32 . 33 33 33 33 33 33 33 . 34 34 34 34 34 . 35 . 36 36 36 36 36 36 36 . 37 37 37 37 37 37 . 38 38 38 38 . 39 39 39 39 39 39 39 . 40 40 40 40 40 40 . 41 . 42 42 42 . 43 43 . 44 44 44 44 44 44 '

Refactor

In [12]:
random.seed(9)
text = ". ".join([f"{str(s)} "*randrange(1, 8) for s in np.arange(0, 45)])
# print(text)

# calculate the number of tokens in text
token_count = len(nltk.word_tokenize(text))


max_split_length = 50
init_split_length = 30

n_splits = int(np.ceil(token_count / init_split_length))
# n_splits = int(np.ceil(token_count / max_split_length))
print("n_splits", n_splits)

sent = np.array(nltk.sent_tokenize(text))

# create an array of sentence token lengths
sent_token_lengths = np.array([len(nltk.word_tokenize(s)) for s in sent])
sent_split_lengths = np.array_split(sent_token_lengths, n_splits)
sent_split_indices = np.array_split(np.arange(0, len(sent_token_lengths)), n_splits)

split_dict = {}
for i in range(n_splits):
    chunk_lengths = int(np.sum(sent_split_lengths[i]))
    chunk_indices = list(sent_split_indices[i])

    if i == 0:
        forwards_indices = np.hstack(np.array(sent_split_indices[i+1:], dtype=object))
        forwards_lengths = np.hstack(np.array(sent_split_lengths[i+1:], dtype=object))
        forwards = list(zip(forwards_lengths, forwards_indices))

        for j in forwards:
            if chunk_lengths + j[0] <= max_split_length:
                chunk_lengths += j[0]
                chunk_indices = chunk_indices + [j[1]]
            else:
                break


        # for j in zip(sent_split_lengths[i+1], sent_split_indices[i+1]):
        #     if chunk_lengths + j[0] <= max_split_length:
        #         chunk_lengths += j[0]
        #         chunk_indices = chunk_indices + [j[1]]
        
        split_text = " ".join([s for s in sent[chunk_indices]])
        split_dict[i] = [split_text, chunk_lengths]

    elif i > 0 and i <= n_splits - 2:
        

        backwards_indices = np.hstack(np.array(sent_split_indices[:i], dtype=object))
        backwards_lengths = np.hstack(np.array(sent_split_lengths[:i], dtype=object))

        forwards_indices = np.hstack(np.array(sent_split_indices[i+1:], dtype=object))
        forwards_lengths = np.hstack(np.array(sent_split_lengths[i+1:], dtype=object))

        forwards = list(zip(forwards_lengths, forwards_indices))
        backwards = list(zip(backwards_lengths[::-1], backwards_indices[::-1]))

        for k in range(max_split_length):
            if k % 2 == 0:
                if chunk_lengths + forwards[0][0] <= max_split_length:
                    chunk_lengths += forwards[0][0]
                    chunk_indices = chunk_indices + [forwards[0][1]]
                    forwards.pop(0)
                else:
                    break
            else:
                if chunk_lengths + backwards[0][0] <= max_split_length:
                    chunk_lengths += backwards[0][0]
                    chunk_indices = [backwards[0][1]] + chunk_indices
                    backwards.pop(0)
                else:
                    break

            # if chunk_lengths >= max_split_length:
            #     break

        split_text = " ".join([s for s in sent[chunk_indices]])
        split_dict[i] = [split_text, chunk_lengths]
        
    else:
        for j in zip(sent_split_lengths[i-1][::-1], sent_split_indices[i-1][::-1]):
            if chunk_lengths + j[0] <= max_split_length:
                chunk_lengths += j[0]
                chunk_indices = [j[1]] + chunk_indices
        split_text = " ".join([s for s in sent[chunk_indices]])
        split_dict[i] = [split_text, chunk_lengths]

n_splits 8


In [13]:
split_dict

{0: ['0 0 0 0 . 1 1 1 1 1 . 2 2 2 . 3 3 3 . 4 4 . 5 5 . 6 6 6 6 6 6 6 . 7 7 7 7 7 7 . 8 . 9 9 9 .',
  46],
 1: ['4 4 . 5 5 . 6 6 6 6 6 6 6 . 7 7 7 7 7 7 . 8 . 9 9 9 . 10 10 10 10 10 . 11 11 11 11 . 12 12 12 12 12 . 13 . 14 14 14 .',
  50],
 2: ['11 11 11 11 . 12 12 12 12 12 . 13 . 14 14 14 . 15 15 15 15 15 . 16 16 16 16 16 . 17 17 17 17 17 17 . 18 . 19 19 19 19 19 19 .',
  45],
 3: ['17 17 17 17 17 17 . 18 . 19 19 19 19 19 19 . 20 20 20 20 . 21 21 . 22 22 22 22 22 22 . 23 23 23 23 . 24 24 24 24 24 24 . 25 25 25 25 .',
  48],
 4: ['21 21 . 22 22 22 22 22 22 . 23 23 23 23 . 24 24 24 24 24 24 . 25 25 25 25 . 26 26 . 27 27 . 28 28 . 29 . 30 . 31 31 . 32 32 32 32 32 .',
  49],
 5: ['27 27 . 28 28 . 29 . 30 . 31 31 . 32 32 32 32 32 . 33 33 33 33 33 33 33 . 34 34 34 34 34 . 35 . 36 36 36 36 36 36 36 . 37 37 37 37 37 37 .',
  50],
 6: ['34 34 34 34 34 . 35 . 36 36 36 36 36 36 36 . 37 37 37 37 37 37 . 38 38 38 38 . 39 39 39 39 39 39 39 . 40 40 40 40 40 40 . 41 .',
  45],
 7: ['36 36 36 36 36 36

# Below is working

In [9]:
# text = text3 * 5

random.seed(9)
text = ". ".join([f"{str(s)} "*randrange(1, 8) for s in np.arange(0, 45)])
print(text)
# text = ". ".join([str(s) for s in np.arange(0, 19)])

# calculate the number of tokens in text
token_count = len(nltk.word_tokenize(text))


max_split_length = 50
init_split_length = 30

n_splits = int(np.ceil(token_count / init_split_length))
# n_splits = int(np.ceil(token_count / max_split_length))
print("n_splits", n_splits)

sent = np.array(nltk.sent_tokenize(text))

# create an array of sentence token lengths
sent_token_lengths = np.array([len(nltk.word_tokenize(s)) for s in sent])

sent_split_lengths = np.array_split(sent_token_lengths, n_splits)

sent_split_indices = np.array_split(np.arange(0, len(sent_token_lengths)), n_splits)

split_dict = {}
for i in range(n_splits):
    print("i: ", i)
    chunk_len = chunk_len_orig = int(np.sum(sent_split_lengths[i]))
    # print("chunk_len: ", chunk_len)
    chunk_indices = list(sent_split_indices[i])
    if i == 0:
        for j in zip(sent_split_lengths[i+1], sent_split_indices[i+1]):
            if chunk_len + j[0] <= max_split_length:
                chunk_len += j[0]
                chunk_indices = chunk_indices + [j[1]]
        
        split_text = " ".join([s for s in sent[chunk_indices]])
        split_dict[i] = [split_text, chunk_len]

    elif i > 0 and i <= n_splits - 2:
        

        backwards_indices = np.hstack(np.array(sent_split_indices[:i], dtype=object))
        backwards_lengths = np.hstack(np.array(sent_split_lengths[:i], dtype=object))

        forwards_indices = np.hstack(np.array(sent_split_indices[i+1:], dtype=object))
        forwards_lengths = np.hstack(np.array(sent_split_lengths[i+1:], dtype=object))

        forwards = list(zip(forwards_lengths, forwards_indices))
        backwards = list(zip(backwards_lengths[::-1], backwards_indices[::-1]))

        for k in range(max_split_length):
            if k % 2 == 0:
                if chunk_len + forwards[0][0] <= max_split_length:
                    chunk_len += forwards[0][0]
                    chunk_indices = chunk_indices + [forwards[0][1]]
                    forwards.pop(0)
            else:
                if chunk_len + backwards[0][0] <= max_split_length:
                    chunk_len += backwards[0][0]
                    chunk_indices = [backwards[0][1]] + chunk_indices
                    backwards.pop(0)

            if chunk_len >= max_split_length:
                break

        split_text = " ".join([s for s in sent[chunk_indices]])
        split_dict[i] = [split_text, chunk_len]
        
    else:
        for j in zip(sent_split_lengths[i-1][::-1], sent_split_indices[i-1][::-1]):
            if chunk_len + j[0] <= max_split_length:
                chunk_len += j[0]
                chunk_indices = [j[1]] + chunk_indices
        split_text = " ".join([s for s in sent[chunk_indices]])
        split_dict[i] = [split_text, chunk_len]

0 0 0 0 . 1 1 1 1 1 . 2 2 2 . 3 3 3 . 4 4 . 5 5 . 6 6 6 6 6 6 6 . 7 7 7 7 7 7 . 8 . 9 9 9 . 10 10 10 10 10 . 11 11 11 11 . 12 12 12 12 12 . 13 . 14 14 14 . 15 15 15 15 15 . 16 16 16 16 16 . 17 17 17 17 17 17 . 18 . 19 19 19 19 19 19 . 20 20 20 20 . 21 21 . 22 22 22 22 22 22 . 23 23 23 23 . 24 24 24 24 24 24 . 25 25 25 25 . 26 26 . 27 27 . 28 28 . 29 . 30 . 31 31 . 32 32 32 32 32 . 33 33 33 33 33 33 33 . 34 34 34 34 34 . 35 . 36 36 36 36 36 36 36 . 37 37 37 37 37 37 . 38 38 38 38 . 39 39 39 39 39 39 39 . 40 40 40 40 40 40 . 41 . 42 42 42 . 43 43 . 44 44 44 44 44 44 
n_splits 8
i:  0
i:  1
i:  2
i:  3
i:  4
i:  5
i:  6
i:  7


In [10]:
split_dict

{0: ['0 0 0 0 . 1 1 1 1 1 . 2 2 2 . 3 3 3 . 4 4 . 5 5 . 6 6 6 6 6 6 6 . 7 7 7 7 7 7 . 8 . 9 9 9 .',
  46],
 1: ['4 4 . 5 5 . 6 6 6 6 6 6 6 . 7 7 7 7 7 7 . 8 . 9 9 9 . 10 10 10 10 10 . 11 11 11 11 . 12 12 12 12 12 . 13 . 14 14 14 .',
  50],
 2: ['11 11 11 11 . 12 12 12 12 12 . 13 . 14 14 14 . 15 15 15 15 15 . 16 16 16 16 16 . 17 17 17 17 17 17 . 18 . 19 19 19 19 19 19 . 20 20 20 20 .',
  50],
 3: ['17 17 17 17 17 17 . 18 . 19 19 19 19 19 19 . 20 20 20 20 . 21 21 . 22 22 22 22 22 22 . 23 23 23 23 . 24 24 24 24 24 24 . 25 25 25 25 .',
  48],
 4: ['21 21 . 22 22 22 22 22 22 . 23 23 23 23 . 24 24 24 24 24 24 . 25 25 25 25 . 26 26 . 27 27 . 28 28 . 29 . 30 . 31 31 . 32 32 32 32 32 .',
  49],
 5: ['27 27 . 28 28 . 29 . 30 . 31 31 . 32 32 32 32 32 . 33 33 33 33 33 33 33 . 34 34 34 34 34 . 35 . 36 36 36 36 36 36 36 . 37 37 37 37 37 37 .',
  50],
 6: ['34 34 34 34 34 . 35 . 36 36 36 36 36 36 36 . 37 37 37 37 37 37 . 38 38 38 38 . 39 39 39 39 39 39 39 . 40 40 40 40 40 40 . 41 . 42 42 42 .',
  49]

In [ ]:
print(backward_indices)
print(forwards_indices)
print(chunk_indices)

In [ ]:
len(nltk.word_tokenize(split_dict[0][0]))

In [ ]:
chunk_indices

In [ ]:
backward_indices

**Working!**

In [ ]:
# calculate the number of tokens in text
token_count = len(nltk.word_tokenize(text))


max_split_length = 500

n_splits = int(np.ceil(token_count / max_split_length))

sent = np.array(nltk.sent_tokenize(text))

# create an array of sentence token lengths
sent_token_lengths = np.array([len(nltk.word_tokenize(s)) for s in sent])

sent_split_lengths = np.array_split(sent_token_lengths, n_splits)

sent_split_indices = np.array_split(np.arange(0, len(sent_token_lengths)), n_splits)

split_dict = {}
for i in range(n_splits):
    chunk_len = int(np.sum(sent_split_lengths[i]))
    chunk_indices = list(sent_split_indices[i])
    if i == 0:
        for j in zip(sent_split_lengths[i+1], sent_split_indices[i+1]):
            if chunk_len + j[0] <= max_split_length:
                chunk_len += j[0]
                chunk_indices = chunk_indices + [j[1]]
        
        split_text = " ".join([s for s in sent[chunk_indices]])
        split_dict[i] = [split_text, chunk_len]

    else:
        for j in zip(sent_split_lengths[i-1][::-1], sent_split_indices[i-1][::-1]):
            if chunk_len + j[0] <= max_split_length:
                chunk_len += j[0]
                chunk_indices = [j[1]] + chunk_indices
        split_text = " ".join([s for s in sent[chunk_indices]])
        split_dict[i] = [split_text, chunk_len]

In [ ]:
split_text = split_dict[0][0]
split_text